In [ ]:
from __future__ import division, print_function
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import nengo
from nengo.utils.matplotlib import rasterplot
import scipy
import scipy.signal
from scipy.io.wavfile import read as readwav
from IPython.display import Audio

# How to build a brain
# with Python

### Trevor Bekolay

note: terminator / spuan background?

My name is Trevor, and I'm a theoretical neuroscientist.
That means that I use computers
and math to build models of the brain.
When someone asks what I do, and I tell them this,
I get one of three reactions.
The first is very pleasant:
we'll share in the mysteries of the brain,
"Why do we need sleep, and why do we dream?"
"Why am I so terrible at remembering names, but I'm good with numbers?"
"Why do some people see the dress as white and gold, what's wrong with them?"
The second reaction I often get starts off like the first:
"I watched a movie about this..."
but then ends with
"it's called Terminator. Are you making SkyNet?!"
The third reaction is probably the most common:
"Sorry, what exactly do you do?"
Very fair question.

The reality is that I spend most of my days
doing what most of you do:
programming, in Python as much as possible. 
Today, I want to share with you
how I and other theoretical neuoscientists
are using Python to model the brain.
And in doing so, I hope to teach you
a bit about how your brain works,
on a meaningful level.

Feynman chalkboard pic

First, I want to start on a bit of philosophical note.
Why do we want to simulate or build models of the brain?
The SkyNet reference here is a fair one;
this is one possible eventuality of this kind of research.
But the reason most if not all theoretical neuroscientsts
want to build a brain is to understand it.
We've learned a lot about the brain by
dissecting it and probing it,
but imagine you were to take apart
a laptop or even a radio with no knowledge
of electronics and circuits.
It would take a long time to figure out how it works,
yet we've come up with these incredibly intricate designs
starting from first principles of electronics
and having a goal;
like in the radio,
you read EM waves and replicate the sounds encoded in them.
There are many ways to design a radio,
and I think you can agree that they're all functionally the same.
Yet, so far we've been understanding the brain with experiments.
Theoretical neuroscientists go the other way;
starting from the fundamental biological parts --
or at least models of those parts --
can we replicate the function of the brain?

brain basics

zoom in, just a bunch of neurons

Let's first talk about those fundamental biological parts,
and in doing so we'll start writing Python code, finally!

The human brain is made up of about 85 billion
interconnected cells called neurons,
and about the same number of support cells
called glia. Glia are super interesting,
but I will completely ignore them
for the remainder of this talk,
as neurons appear to be the primary workhorses
of the brain; you can stimulate neurons
in the motor regions of the brain
to cause muscle contractions,
stimulate visual areas to consciously
see things that aren't there, and so on.

What is a neuron and what does it do?

takes in input (from dendrites)

produces an output (from axon)

`neuron(*dendrites) -> axon`

NOTE: Just one output!

What is this? A function!

So what do neurons do.
Well, it's complicated,
but a useful simplification --
for us programmer types anyway --
is to think of neurons as computing a function.

It receives input from other neurons
through its dendrites.
It then uses that input to produce
some output.
And although it is not typically our way as Pythonistas,
let's give these things types and units anyway.
Our dendritic inputs are currents in amps (typically pico or microamps),
and our output is voltage in volts (typically millivolts).

Action potentials

J -> neuron -> spike -> J -> neuron

This is an example of the output of a neuron.
We can measure this in real neurons with electrodes.
Most of what's going on here isn't important at all;
what's actually important (zoom in)
are these spikes of high voltage;
their technical name is an action potential,
but I'll call them spikes.
Output only actually happens
when a neuron spikes.
When a neuron spikes,
releases tiny chemicals,
which causes current to flow into
the neurons that it is connected to.
This is the basic idea:
current flows into a neuron,
and if enough current flows in,
then the neuron spikes,
causing current to flow into downstream neurons.

Neurons are weird

put in some odd neuron things; probably from Izhikevich

One way to spend your life as a theoretical neuroscientist:
making neuron types

So, I told you that a neuron is essentially a function.
If you want to be very precise,
that function is actually quite complex;
but this is where brain building gets its start,
is emulating very accurately the activity
of one or two neurons.
We can call this "neuron modelling."

Introduce NEURON

One of the first pieces of software
to help with making neuron models
is called NEURON, which has existed in
some form since about 1984.
It's writting in a mix of C, C++ and Fortran.
Most students when they're exposed to
theoretical neuroscience for the first time
will use NEURON's GUI,
which by this point is showing its age.
(Quick demo)

But, it is very much still a relevant tool;
it is the software that allows the Blue Brain project,
for example, to run large networks on
the IBM Blue Gene supercomputer.
And, since it is mostly C,
it's very natural to hook up to Python.

Show PyNeuron example

*Figure out what kind of example to show*

Introduce Brian

Here's a familiar story:
there's a C project with Python bindings,
which is initially super cool,
but quickly is such a tease;
it's nice to be able to use it in Python,
but it would also be nice
if it were all in Python so it was easy to edit.
It seems to have happened in all domains,
and theoretical neuroscience is no exception.

Brian is a neural simulator written entirely in Python.

Show Brian example

Here, we use Brian to do the exact same model
as before, but using Brian.
*Something about ease of use, speed?*

Notable simulators with Python bindings

I've just shown you two neural simulators,
the two that I consider the most notable,
but in fact there are several others.
There are approximately as many neural simulators
as there are labs making these models;
as the saying goes,
simulators are like toothbrushes:
everyone's got one,
but no one likes to use someone else's.

1 billion dollar question:
if you were to simulate 100 billion of these,
would you get a human brain?

Every neuron has 10,000 connections.
Neurons (functions) are important,
but connections (inputs) are more important

Now, we've simulated a few neurons.
If you're like me, you're probably not satisfied.
I want to know how the *brain* works,
I want to build a *brain*,
not simulate a neuron!
The brain is just 85 billion neurons.

So, let me ask you in the audience.
Hands up if you think we can
essentially copy the neuron from above
85 billion times,
and we'll get something that resembles the brain?

Very few hands (likely). That's good;
every neuron is connected to, on average,
10,000 other neurons.
That means that, while there are 85 billion neurons,
there are on the order of a quadrillion connections between neurons.
This seems to be where we should be concentrating out efforts.

Okay, back to the audience.
Do you think we can build a brain by
taking 85 billion neurons,
and making a quadrillion random connections,
show of hands?
How about this:
we do some detailed measurements of each neuron,
and figure out where the connections are coming from,
and how strong they are;
then, instead of making a quadrillion totally random connections,
we make connections that match the statistics of real brains.
How many think we'll get brain-like behavior?
Okay, last try.
There's a lot of learning happening in the brain,
which is physically manifest as changing the strength
of the connections between neurons based on their activity.
What if we had those statistically matched quadrillion connections,
and also learning was happening in a biologically consistent way.
How many think we'll get brain-like behavior?

Another approach: Neural Engineering Framework

A theory for how to build a brain

In the remaining time, let me introduce you to
the Neural Engineering Framework,
and a Python library called Nengo,
which is, in my opinion,
a satisfying way to build brain models.

Block diagram

The basic idea is this:
each neuron is not that important,
but collectively, a group of neurons
(which I will call an "ensemble")
represents some information,
like a number of a list of numbers,
and the connection from one group of neurons
to another group
transforms that information
in some way that can be expressed with mathematics.

So, say that this ensemble represents
a point in two-dimensional space -- so two numbers.
Then, we could have this ensemble
represent the distance from the origin to that point
with the distance formula.

Analogy: Humans talk in base 10

Computers talk in base 2

Give quick rundown of how to convert

How does this work? Well, like with neuron modelling,
I only have time to scratch the surface,
but here's an analogy to give you a sense of what's going on.

As programmers, we should be pretty familiar with
binary numbers, since that's how computer represent information.

Briefly, *explain how to convert decimal to binary*.

Brains talk in base neural activity

There's a similar analogy between numbers and how
we represent information in the NEF.
In this case, *explain how to convert decimal to NEF*.

In the NEF, neurons aren't the target:
they're just what we use to implement math.

Then, if you can describe a system in math (dynamical systems),
you can build a brain to implement that math.

This is how I spend my life as a theoretical neuroscientist:
solving problems with math and then implementing it
in the NEF.

Essentially this gives us a way to ignore how weird neurons are.
Instead, we come up with the math to solve a problem,
and compile that down to a brain model that
implements that math using neurons.

Introduce Nengo

Let's see how this works, using Nengo.
*E.g.*

Show Nengo example (2D eye position tracking)

*Model*

Hypotheses from this model

Predictions from this model

*Hypotheses and predictions*

We've only scratched the surface,
but I hope you can get a sense
for what we do.
We think about:

- what information is needed to do something?
- how is that information manipulated?

*More*

Using just the tools I showed in the Nengo example,
and around 8 years of collective work,
you can make Spaun

*More*

Awesome things I didn't have time for:

- learning
- neuromorphic hardware
- many more

There's tons of interesting new things happening,
but they are unfortunately quite new.
The field is young, and somewhat undocumented.
We're trying our best with Nengo,
but there are so many things that
programmers could help with;
it just takes a fair bit of work.

Thanks to

- Nengo contribs
- Others

Pointers to learn more

- links
- to
- Python in Neuroscience research topic
- NEF / Nengo things, examples

*More*